In [5]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.pipeline import make_pipeline

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

from bs4 import BeautifulSoup
from langdetect import detect
from urllib.parse import urlsplit

from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Pre Processing

In [6]:
df = pd.read_csv("../Dataset/IMDB Dataset.csv")
df.head()
df = df.rename(columns={'review': 'OriginalReviews'})
df = df.rename(columns={'sentiment': 'OutputSentiment'})
df_subset = df.sample(n=5000).reset_index(drop=True)
df_subset.head()
df_subset['OutputSentiment'].value_counts()

OutputSentiment
negative    2512
positive    2488
Name: count, dtype: int64

In [7]:
#lowercase
df_subset["OriginalReviews"]=df_subset["OriginalReviews"].apply(lambda x:x.lower())

#Remove punctuations
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].str.replace('[{}]'.format(string.punctuation), ' ')

# Remove numbers from the 'OriginalReviewss' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].str.replace('\d+', '')

In [8]:
# Function to remove stopwords from a text
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

# Apply the remove_stopwords function to the 'OriginalReviews' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(remove_stopwords)

In [9]:
def remove_urls(text):
    # Define a regular expression pattern to match URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')

    # Find all matches in the text
    urls = re.findall(url_pattern, text)

    # Remove URLs from the text
    text_without_urls = re.sub(url_pattern, '', text)

    return text_without_urls

# Example usage
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(remove_urls)

In [10]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

df_subset["OriginalReviews"] = df_subset["OriginalReviews"].apply(remove_html_tags)

C:\Users\sdidd\AppData\Local\Temp\ipykernel_23280\3731208440.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [11]:
def clean_text(text):
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(clean_text)

In [12]:
def remove_extra_whitespaces(text):
    # Use regular expression to replace multiple whitespaces with a single space
    return re.sub(r'\s+', ' ', text).strip()

df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(remove_extra_whitespaces)

In [13]:
def filter_non_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

# Create a boolean mask for non-English OriginalReviewss
mask = df_subset['OriginalReviews'].apply(filter_non_english)

# Create a new DataFrame containing only English OriginalReviewss
df_subset = df_subset[mask]

In [14]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to get the part of speech for WordNet lemmatizer
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if the part of speech is not found

# Function to lemmatize a text
def lemmatize_text(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tags]
    return ' '.join(lemmatized_tokens)

# Apply lemmatization to the 'text' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(lemmatize_text)

In [15]:
df_subset

,OriginalReviews,OutputSentiment
0,last believe movie boy make hal roach also las...,positive
1,crush pleasant enough 40something friend roman...,negative
2,first seem topical romance movie girl meet boy...,positive
3,really think excellent job nothing wrong nt kn...,positive
4,seem bizarre someone read script think funny m...,negative
...,...,...
4995,idea fact film present remember situation acce...,positive
4996,despite occasionally original touch like virtu...,negative
4997,main achievement film though racially unipolar...,positive
4998,agree user comment two main role well act guy ...,negative


In [16]:
df_subset.to_csv("../csv/Preprocessed_data.csv",index=False)

## Feature Extraction Using TF-IDF

In [17]:
preprocessed = pd.read_csv('../csv/Preprocessed_data.csv')

In [18]:
num_features_to_keep = 13000

# Create a pipeline with TfidfVectorizer and SelectKBest
pipeline = make_pipeline(TfidfVectorizer(), SelectKBest(f_classif, k=num_features_to_keep))

# Fit and transform your data
X_transformed = pipeline.fit_transform(preprocessed['OriginalReviews'], preprocessed['OutputSentiment'])

# Get the selected feature names
selected_feature_names = pipeline.named_steps['tfidfvectorizer'].get_feature_names_out()[pipeline.named_steps['selectkbest'].get_support()]

# Create a DataFrame with the selected features
selected_features_df = pd.DataFrame(X_transformed.toarray(), columns=selected_feature_names)

# Concatenate the existing DataFrame with the new selected features DataFrame
tfidf_df_13k = pd.concat([preprocessed, selected_features_df], axis=1)

tfidf_df_13k.head()

tfidf_df_13k.to_csv("../csv/tfidf_df_13k.csv")

## CONNOTATIONS

In [19]:
# Download the VADER lexicon (run this once)
nltk.download('vader_lexicon')
delimiter = '\t'

# Read the text file into a DataFrame
positive = pd.read_csv(r'..\Connotations\positive-words.txt', sep=delimiter, names=['words'])
negative = pd.read_csv(r'..\Connotations\negative-words.txt', sep=delimiter, names=['words'])
connotations = pd.read_csv(r"..\Connotations\connotations.csv")

word_emotion_map = dict(zip(connotations['word'], connotations['emotion']))

def update_counts(review):
    positive_count = sum(1 for word in review.split() if word in word_emotion_map and word_emotion_map[word] == 'positive')
    negative_count = sum(1 for word in review.split() if word in word_emotion_map and word_emotion_map[word] == 'negative')
    return positive_count, negative_count

tfidf_df_13k[['Positive_Connotation_Count', 'Negative_Connotation_Count']] = tfidf_df_13k['OriginalReviews'].apply(update_counts).tolist()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [20]:
# Load positive and negative words from files
positive_words_df = pd.read_csv(r'..\Connotations\positive-words.txt', header=None, names=['words'])
negative_words_df = pd.read_csv(r'..\Connotations\negative-words.txt', header=None, names=['words'])

# Convert DataFrame columns to sets
positive_words = set(positive_words_df['words'].tolist())
negative_words = set(negative_words_df['words'].tolist())

# Assuming 'tfidf_df_13k' is your DataFrame
# Define a function to update counts based on positive and negative words
def update_word_counts(review):
    positive_count = sum(1 for word in review.split() if word in positive_words)
    negative_count = sum(1 for word in review.split() if word in negative_words)
    return positive_count, negative_count

# Apply the function to the 'OriginalReviews' column and unpack the result into two new columns
tfidf_df_13k[['Positive_Word_Count', 'Negative_Word_Count']] = tfidf_df_13k['OriginalReviews'].apply(update_word_counts).tolist()

In [21]:
from nltk.sentiment import SentimentIntensityAnalyzer
# Use VADER for sentiment analysis
sid = SentimentIntensityAnalyzer()

def vader_sentiment(review):
    scores = sid.polarity_scores(review)
    return scores['pos'] *100, scores['neg'] * 100

# Apply the function to the 'OriginalReviews' column and unpack the result into two new columns
tfidf_df_13k[['Positive_VADER_Count', 'Negative_VADER_Count']] = tfidf_df_13k['OriginalReviews'].apply(vader_sentiment).tolist()

tfidf_df_13k.to_csv("../csv/tfidf_df_13k_connotations_vader.csv")

In [22]:
text = tfidf_df_13k.iloc[4993]['OriginalReviews']

In [23]:
tfidf_df_13k = pd.read_csv("../csv/tfidf_df_13k.csv")

In [24]:
tfidf_df_13k_connotations = pd.read_csv('../csv/tfidf_df_13k_connotations_vader.csv')

In [25]:
tfidf_df_13k_connotations = tfidf_df_13k_connotations.drop('Unnamed: 0',axis=1)

In [26]:
tfidf_df_13k_connotations

,OriginalReviews,OutputSentiment,01,010,02,06,10,10000,100000,101,...,zugsmith,zula,zulu,zwick,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,last believe movie boy make hal roach also las...,positive,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,31,23,10,5,34.6,7.1
1,crush pleasant enough 40something friend roman...,negative,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,47,38,13,6,21.0,6.6
2,first seem topical romance movie girl meet boy...,positive,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,31,12,4,1,20.5,0.0
3,really think excellent job nothing wrong nt kn...,positive,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,32,18,8,4,42.3,10.3
4,seem bizarre someone read script think funny m...,negative,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,33,38,9,11,25.9,16.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4991,idea fact film present remember situation acce...,positive,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,16,8,4,3,12.8,11.3
4992,despite occasionally original touch like virtu...,negative,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,34,17,4,5,8.2,10.0
4993,main achievement film though racially unipolar...,positive,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,185,156,26,67,12.4,27.4
4994,agree user comment two main role well act guy ...,negative,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,16,16,3,6,19.9,24.8


In [27]:
df_statistical = tfidf_df_13k_connotations.drop(columns=['OriginalReviews','Positive_Connotation_Count','Negative_Connotation_Count','Positive_Word_Count','Negative_Word_Count','Positive_VADER_Count','Negative_VADER_Count'], axis=1)
df_statistical.head()

,OutputSentiment,01,010,02,06,10,10000,100000,101,1010,...,zoomouts,zooms,zorro,zuber,zucco,zuckerman,zugsmith,zula,zulu,zwick
0,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
label = LabelEncoder()
df_statistical['OutputSentiment'] = label.fit_transform(df_statistical['OutputSentiment'])

## CHI SQAURE

In [29]:
# This will get the top 5000 relavant features out of the sample
chi2_selector = SelectKBest(chi2, k=5000)

# This will transform the dataset i.e, it will reduce the dimensions by just considering the relavant features only
X = df_statistical.drop(columns=['OutputSentiment'])
y = df_statistical['OutputSentiment']
X_5000 = chi2_selector.fit_transform(X, y)

# Get the indices of the selected features
selected_feature_indices = chi2_selector.get_support(indices=True)

# Get the names of the selected features
selected_feature_names = X.columns[selected_feature_indices]

chisq_5k = X[selected_feature_names]
chisq_5k.head()

chisq_5k = pd.concat([chisq_5k,tfidf_df_13k_connotations.iloc[:, -6:]],axis=1)
chisq_5k.head()

,010,06,10,10000,101,1010,10th,110,117,11th,...,zoo,zoom,zorro,zucco,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,31,23,10,5,34.6,7.1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,47,38,13,6,21.0,6.6
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,31,12,4,1,20.5,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,32,18,8,4,42.3,10.3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,33,38,9,11,25.9,16.5


In [30]:
# This will get the top 5000 relavant features out of the sample
chi2_selector = SelectKBest(chi2, k=8000)

# This will transform the dataset i.e, it will reduce the dimensions by just considering the relavant features only
X = df_statistical.drop(columns=['OutputSentiment'])
y = df_statistical['OutputSentiment']
X_8000 = chi2_selector.fit_transform(X, y)

# Get the indices of the selected features
selected_feature_indices = chi2_selector.get_support(indices=True)

# Get the names of the selected features
selected_feature_names = X.columns[selected_feature_indices]

chisq_8k = X[selected_feature_names]
chisq_8k.head()

chisq_8k = pd.concat([chisq_8k,tfidf_df_13k_connotations.iloc[:, -6:]],axis=1)
chisq_8k.head()

,01,010,02,06,10,10000,101,1010,102,105,...,zucco,zuckerman,zulu,zwick,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,31,23,10,5,34.6,7.1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,47,38,13,6,21.0,6.6
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,31,12,4,1,20.5,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,32,18,8,4,42.3,10.3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,33,38,9,11,25.9,16.5


In [31]:
chisq_8k

,01,010,02,06,10,10000,101,1010,102,105,...,zucco,zuckerman,zulu,zwick,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,31,23,10,5,34.6,7.1
1,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,47,38,13,6,21.0,6.6
2,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,31,12,4,1,20.5,0.0
3,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,32,18,8,4,42.3,10.3
4,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,33,38,9,11,25.9,16.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4991,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,16,8,4,3,12.8,11.3
4992,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,34,17,4,5,8.2,10.0
4993,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,185,156,26,67,12.4,27.4
4994,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,16,16,3,6,19.9,24.8


## CHI SQAURE CLASSIFICATION

In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

# Multinomial Naive Bayes Classifier
nb_classifier = MultinomialNB()
nb_scores = cross_val_score(nb_classifier, chisq_8k, y, cv=5)

print("Multinomial Naive Bayes Cross-Validation Scores:")
print(nb_scores)
print("Mean Accuracy:", np.mean(nb_scores))

# k-Nearest Neighbors Classifier
knn_classifier = KNeighborsClassifier()
knn_scores = cross_val_score(knn_classifier, chisq_5k, y, cv=5)

print("\nk-Nearest Neighbors Cross-Validation Scores:")
print(knn_scores)
print("Mean Accuracy:", np.mean(knn_scores))

Multinomial Naive Bayes Cross-Validation Scores:
[0.742      0.74174174 0.75775776 0.76276276 0.74474474]
Mean Accuracy: 0.7498014014014014

k-Nearest Neighbors Cross-Validation Scores:
[0.716      0.68268268 0.7037037  0.6976977  0.68768769]
Mean Accuracy: 0.6975543543543543


In [33]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import cross_val_score, KFold
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression

# # Load your data
# # Assuming X and y are your features and target variables

# # Initialize models
# svm_model = SVC(kernel='linear')  # Linear SVM
# logistic_model = LogisticRegression()

# # Initialize KFold cross-validation
# kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# # Perform 5-fold cross-validation for SVM
# svm_scores = cross_val_score(svm_model, chisq_5k, y, cv=kfold)

# # Perform 5-fold cross-validation for Logistic Regression
# logistic_scores = cross_val_score(logistic_model, chisq_8k, y, cv=kfold)

# # Display the cross-validation scores
# print("SVM Cross-validation scores:", svm_scores)
# print("Logistic Regression Cross-validation scores:", logistic_scores)

# # Optionally, you can calculate mean and standard deviation of the scores
# print("SVM Mean Accuracy:", np.mean(svm_scores))
# print("SVM Standard Deviation of Accuracy:", np.std(svm_scores))
# print("Logistic Regression Mean Accuracy:", np.mean(logistic_scores))
# print("Logistic Regression Standard Deviation of Accuracy:", np.std(logistic_scores))

In [34]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import StratifiedKFold

# Assuming chisq_8k has features and y is the output

# Encode categorical labels if needed
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(chisq_8k, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a simple neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=1)

# Evaluate the model on the test set
y_pred = (model.predict(X_test_scaled) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)

print("Neural Network Accuracy on Test Set:", accuracy)




Epoch 1/10


125/125 [==============================] - 3s 12ms/step - loss: 0.3832 - accuracy: 0.8086
Epoch 2/10
125/125 [==============================] - 1s 11ms/step - loss: 0.0289 - accuracy: 0.9897
Epoch 3/10
125/125 [==============================] - 1s 11ms/step - loss: 0.0027 - accuracy: 0.9997
Epoch 4/10
125/125 [==============================] - 2s 13ms/step - loss: 8.3753e-04 - accuracy: 1.0000
Epoch 5/10
125/125 [==============================] - 1s 11ms/step - loss: 4.5536e-04 - accuracy: 1.0000
Epoch 6/10
125/125 [==============================] - 1s 10ms/step - loss: 2.9247e-04 - accuracy: 1.0000
Epoch 7/10
125/125 [==============================] - 1s 10ms/step - loss: 2.0422e-04 - accuracy: 1.0000
Epoch 8/10
125/125 [==============================] - 1s 11ms/step - loss: 1.5008e-04 - accuracy: 1.0000
Epoch 9/10
125/125 [==============================] - 1s 12ms/step - loss: 1.1486e-04 - accuracy: 1.0000
Epoch 10/10
32/32 [==============================] - 0s 2ms/st

In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

# Assuming chisq_8k has features and y is the output

# Encode categorical labels if needed
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(chisq_8k, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a simple neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    X_train_scaled, y_train, 
    epochs=50, batch_size=32, 
    validation_split=0.1,  # Using a portion of training set for validation
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model on the test set
y_pred = (model.predict(X_test_scaled) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)

print("Neural Network Accuracy on Test Set:", accuracy)


Epoch 1/50
113/113 [==============================] - 2s 13ms/step - loss: 0.3557 - accuracy: 0.8270 - val_loss: 0.1645 - val_accuracy: 0.9350
Epoch 2/50
113/113 [==============================] - 1s 12ms/step - loss: 0.0213 - accuracy: 0.9930 - val_loss: 0.1485 - val_accuracy: 0.9375
Epoch 3/50
113/113 [==============================] - 1s 12ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.1565 - val_accuracy: 0.9400
Epoch 4/50
113/113 [==============================] - 1s 11ms/step - loss: 7.7599e-04 - accuracy: 1.0000 - val_loss: 0.1610 - val_accuracy: 0.9400
Epoch 5/50
32/32 [==============================] - 0s 2ms/step
Neural Network Accuracy on Test Set: 0.93
